In [2]:
!pip install chromadb langchain tiktoken sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (py

In [3]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 8.3 MB/s eta 0:00:00


In [4]:
!pip install pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.6 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140082 sha256=174601a1f0670a7bc5b3c2195b20e1555cfcfc3dcd4941f895b432bfafecbbc0
  Stored in directory: /root/.cache/pip/wheels/4e/c1/68/f7bd0a8f514661f76b5cbe3b5f76e0033d79f1296012cbbf72
Successfully built pdfminer


In [5]:
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
def convertMultiple(fileName):
    text = convert(fileName) #get string of text content of pdf
    textFilename = "Resume.txt"
    textFile = open(textFilename, "w") #make text file
    textFile.write(text) #write text to text file

convertMultiple('/content/Sneha_Resume_AI_NLP.pdf')

In [6]:
# Loop through all TEXT files in a folder
# Pull out all text between two anchors: "New York State Real Property Law" & "common elements of the property."
import re
import os

#myRegex=re.compile("Languages[\s\S]*?Libraries",re.DOTALL)
myRegex = re.compile("Languages(.*)Libraries", re.DOTALL)
object=open('Resume.txt')
Text=object.read()
for subText in myRegex.findall(Text):
    skills = subText
    print(skills)

object.close()

: Python, Apex, Java, C, C++, MySQL, JavaScript, PHP, HTML5, CSS3, Dart



In [7]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


In [8]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

import os

In [9]:
api_key = input('Please enter api key - ')

Please enter api key - sk-proj-7UOHdDgjJ5XTgUiF7maFT3BlbkFJA8826OZzZoxYCSSnIrBO


In [10]:
# setting api key as environment variable
os.environ['OPENAI_API_KEY'] = api_key

In [11]:
# Document Loading
loader = CSVLoader(file_path="job_skills.csv")
data = loader.load()

for record in data[:2]:
    print(record)

page_content='job_link: https://www.linkedin.com/jobs/view/senior-machine-learning-engineer-at-jobs-for-humanity-3804053819\njob_skills: Machine Learning, Programming, Python, Scala, Java, Data Engineering, Distributed Computing, Statistical Modeling, Optimization, Data Pipelines, Cloud Computing, DevOps, Software Development, Data Gathering, Data Preparation, Data Visualization, Machine Learning Frameworks, scikitlearn, PyTorch, Dask, Spark, TensorFlow, Distributed File Systems, Multi node Database Paradigms, Open Source ML Software, Responsible AI, Explainable AI' metadata={'source': 'job_skills.csv', 'row': 0}
page_content='job_link: https://www.linkedin.com/jobs/view/principal-software-engineer-ml-accelerators-at-aurora-3703455068\njob_skills: C++, Python, PyTorch, TensorFlow, MXNet, CUDA, OpenCL, OpenVX, Halide, SIMD programming models, MLspecific accelerators, Linux/unix environments, Deep learning frameworks, Computer vision deep learning models, ML software and hardware technol

In [12]:
# Document Splitting
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(data)

In [12]:
# Embeddings
embeddings_model = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
persist_directory = './chroma/'

In [69]:
!rm -rf ./chroma  # remove old database files if any

In [70]:
# Setting up vectorstore for storing the embeddings
db = Chroma.from_documents(texts, embedding=embeddings_model, persist_directory=persist_directory)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
db = Chroma(persist_directory="/content/drive/MyDrive/chroma", embedding_function=embeddings_model)

In [71]:
print(db._collection.count())

12217


In [72]:
query='Find the job that requires any of the following skills ' + skills
docs = db.similarity_search(query,k=3)
print(len(docs))

3


In [73]:
print(query)

Find the job that requires any of the following skills : Python, Apex, Java, C, C++, MySQL, JavaScript, PHP, HTML5, CSS3, Dart



In [74]:
docs

[Document(page_content="job_link: https://ca.linkedin.com/jobs/view/senior-data-engineer-at-clickjobs-io-3802228415\njob_skills: Data engineering, Architecture, ETL/ELT, Data modeling, Data transformation, Business intelligence, Data security, Master data management, Python, Informatica, Fivetran, Snowflake, Cloud data lakes, Delta lakes, CI/CD, DevOps, ERP systems, Apex, Onbase, Command, Analytical skills, Communication skills, Attention to detail, Organizational skills, Ability to adapt to evolving technologies, Proactive problemsolving, Continuous learning, Bachelor's degree in Computer Science or Information Technology", metadata={'row': 4662, 'source': 'job_skills.csv'}),
 Document(page_content='job_link: https://www.linkedin.com/jobs/view/software-engineer-iii-data-modernization-at-clickjobs-io-3800393742\njob_skills: Software Engineering, Data Engineering, Java, Python, NodeJS, React, Spark, Flink, Storm, Kafka, AWS, EMR, Glu, Lambda, MSK, RDS, Document DB, Apache Aurora, Postgr

In [75]:
retreiver = db.as_retriever()
doc = retreiver.get_relevant_documents(query)
print(len(doc))

4


In [76]:
doc[3].page_content

"job_link: https://www.linkedin.com/jobs/view/lead-data-engineer-python-java-or-scala-at-jobs-for-humanity-3792329863\njob_skills: Data Engineering, Agile Methodology, Data Modeling, Machine Learning, Microservices, Java, Scala, Python, RDBMS, NoSQL, Cloudbased Data Infrastructures, Cloud Computing, Hadoop, Kafka, Spark, Realtime Data, Streaming Applications, Cassandra, Redshift, Snowflake, UNIX/Linux, Shell Scripting, Agile Engineering Practices, Bachelor's Degree, 6+ years in Application Development, 2+ years in Big Data Technologies, 2+ years in Data Modeling, 1+ year in Cloud Computing"

In [77]:
llm = ChatOpenAI(model_name='gpt-4o', temperature=0)

In [78]:
# first initialize the large language model
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retreiver,
    memory=memory
)

In [79]:
result = qa({"question": query})

In [80]:
print(len(result))

3


In [81]:
print(result.keys())

dict_keys(['question', 'chat_history', 'answer'])


In [82]:
print(result['answer'])

Based on the provided job descriptions, the following jobs require some of the skills you mentioned:

1. **Senior Data Engineer at ClickJobs.io**
   - Skills required: Python, Apex

   [Job Link](https://ca.linkedin.com/jobs/view/senior-data-engineer-at-clickjobs-io-3802228415)

2. **Software Engineer III - Data Modernization at ClickJobs.io**
   - Skills required: Java, Python

   [Job Link](https://www.linkedin.com/jobs/view/software-engineer-iii-data-modernization-at-clickjobs-io-3800393742)

3. **Software Engineer III - Full Stack Data Management at ClickJobs.io**
   - Skills required: Java, Python

   [Job Link](https://www.linkedin.com/jobs/view/software-engineer-iii-full-stack-data-management-at-clickjobs-io-3799885949)

4. **Lead Data Engineer - Python, Java, or Scala at Jobs for Humanity**
   - Skills required: Java, Python

   [Job Link](https://www.linkedin.com/jobs/view/lead-data-engineer-python-java-or-scala-at-jobs-for-humanity-3792329863)

None of the job descriptions me

Hybrid Search

In [13]:
!pip install "cassio>=0.1.7"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 20.6 MB/s eta 0:00:00


In [14]:
!pip install lancedb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 14.0 MB/s eta 0:00:00


In [15]:
# example of using bm25 & lancedb -hybrid serch

from langchain.vectorstores import LanceDB
import lancedb
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# Initialize embeddings
embedding = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [16]:
!pip install rank_bm25

In [17]:
# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k =  2  # Retrieve top 2 results

In [18]:
type(texts)

list

In [19]:
persist_directory = './chroma/'

In [20]:
!rm -rf ./chroma  # remove old database files if any

In [22]:
# Setting up vectorstore for storing the embeddings
db = Chroma.from_documents(texts, embedding=embedding, persist_directory=persist_directory)

In [23]:
print(db._collection.count())

12217


In [24]:
retreiver = db.as_retriever()

In [25]:
# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retreiver],
                                       weights=[0.4, 0.6])

# Example customer query
query='Find the job that requires any of the following skills ' + skills
docs = db.similarity_search(query,k=3)
print(len(docs))


# Retrieve relevant documents/products
docs = ensemble_retriever.get_relevant_documents(query)

3


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [26]:
docs[0].page_content

'job_link: https://uk.linkedin.com/jobs/view/senior-data-engineer-at-apexon-3796750607\njob_skills: Java, C/C++, Python, Scala, Spark, Linux, Shell scripting, TDD (JUnit), Maven, Gradle, Ant, Continuous delivery, Continuous deployment, Data structures, Algorithms, Software design, Core programming concepts, Fullstack development, Multitasking, Stakeholder management, Teamwork, Communication, Finance, Problemsolving, Product development, Data modeling, Data visualization, Model mapping, Data Lake, PURE, Alloy'

In [27]:
llm = ChatOpenAI(model_name='gpt-4o', temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [28]:
# first initialize the large language model
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retreiver,
    memory=memory
)

In [29]:
result = qa({'question':query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [30]:
print(result['answer'])

Based on the provided context, the jobs that require any of the specified skills (Python, Java, C, C++) are:

1. **Senior Data Engineer at Apexon**
   - Skills: Java, C/C++, Python, Scala, Spark, Linux, Shell scripting, TDD (JUnit), Maven, Gradle, Ant, Continuous delivery, Continuous deployment, Data structures, Algorithms, Software design, Core programming concepts, Fullstack development, Multitasking, Stakeholder management, Teamwork, Communication, Finance, Problemsolving, Product development, Data modeling, Data visualization, Model mapping, Data Lake, PURE, Alloy
   - [Job Link](https://uk.linkedin.com/jobs/view/senior-data-engineer-at-apexon-3796750607)

2. **Data Scientist at Apex Systems**
   - Skills: Python, Machine learning, Deep learning, Neural networks, Analytical skills, Problemsolving skills, Communication skills, Docker, Kubernetes, Geospatial data, AWS, Azure, Security+ Certification, Databases, SQL, Databricks
   - [Job Link](https://www.linkedin.com/jobs/view/data-s

In [31]:
!pip install gradio


In [32]:
import gradio as gr


In [33]:
# creating the function
def func(question):
  result = qa({'question' : question})
  return result['answer']

In [36]:
# creating the interface
app = gr.Interface(fn=func, inputs = [ 'text'], outputs = 'textbox', title = 'Question Answering bot', theme = 'dark-grass', description = 'Input question, then get answers!')

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:983: UserWarning: Cannot load dark-grass. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark-grass (Request ID: Root=1-66620ac7-4f1efa813395a28c3e15b53c;c5930faf-2d55-4688-95e4-76078e0f33d2)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


In [37]:
# launching the app
app.launch(inline=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cd56918840a2fff32f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [59]:
def func(fileName):
    text = convert(fileName) #get string of text content of pdf
    '''textFilename = "Resume.txt"
    textFile = open(textFilename, "w") #make text file
    print('***',text)
    textFile.write(text) #write text to text file

    myRegex = re.compile("Languages(.*)Libraries", re.DOTALL)
    object=open('Resume.txt')
    Text=object.read()
    print('+++',Text)'''
    myRegex = re.compile("Languages(.*)Libraries", re.DOTALL)
    for subText in myRegex.findall(text):
      print('++++')
      skills = subText
      print(skills)
    object.close()

    query='Find the job that requires any of the following skills ' + skills
    result = qa({'question' : query})
    return result['answer']

In [ ]:
import gradio as gr
import os
import shutil

def process_file(fileobj):
    print(fileobj)
    path = "/content/" + os.path.basename(fileobj)  #NB*
    shutil.copyfile(fileobj.name, path)
    return func(path)

demo = gr.Interface(
    fn=process_file,
    inputs=[
        "file",
    ],
    title = 'Question Answering bot',
    description = 'Input question, then get answers!',
    outputs="text"
)
demo.launch(server_name='0.0.0.0', inline=False, debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://de4dd35031f91ec4c3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
/tmp/gradio/358064d457bec29f678099c5a7b51be49bb15dba/Sneha_Resume_AI_NLP.pdf
++++
: Python, Apex, Java, C, C++, MySQL, JavaScript, PHP, HTML5, CSS3, Dart

/tmp/gradio/358064d457bec29f678099c5a7b51be49bb15dba/Sneha_Resume_AI_NLP.pdf
++++
: Python, Apex, Java, C, C++, MySQL, JavaScript, PHP, HTML5, CSS3, Dart

